In [8]:
# === Built-in ===
import os

# === Third-party ===
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
from scipy.spatial.distance import mahalanobis

# === PyTorch ===
import torch
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights

Classe per Estrazione delle Feature - Task 1-2

In [9]:
class ResNetAvgPool1024Extractor:
    def __init__(self):
        self.model = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.model.eval()
        self.feature = None
        self.model.avgpool.register_forward_hook(self._hook_avgpool)

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])

    def _hook_avgpool(self, module, input, output):
        self.feature = output.squeeze().detach().numpy()

    def extract_feature(self, image_path):
        img = Image.open(image_path).convert('RGB')
        input_tensor = self.transform(img).unsqueeze(0)
        with torch.no_grad():
            _ = self.model(input_tensor)

        feat_2048 = self.feature
        return 0.5 * (feat_2048[::2] + feat_2048[1::2])  # Ridotto a 1024

Estrazione,Salvataggio e Caricamneto delle Feature 

In [10]:
def process_and_save_features(base_folder, subfolders, output_file):
    extractor = ResNetAvgPool1024Extractor()
    all_features, all_filenames, all_labels = [], [], []

    for label in subfolders:
        folder_path = os.path.join(base_folder, label)
        print(f"[INFO] Elaboro cartella: {label}")
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tif')):
                img_path = os.path.join(folder_path, filename)
                features = extractor.extract_feature(img_path)
                if features is not None:
                    all_features.append(features)
                    all_filenames.append(filename)
                    all_labels.append(label)

    np.savez(output_file,
             features=np.array(all_features),
             filenames=np.array(all_filenames),
             labels=np.array(all_labels))
    
    print(f"[SALVATO] Features salvate in {output_file}")
    print(f"[FINE] Totale immagini processate: {len(all_features)}")


def load_features(file_path):
    data = np.load(file_path, allow_pickle=True)
    return data["features"], data["filenames"], data["labels"]

Visualizzazione dei Risultati

In [11]:
def plot_matches(query_path, indices, labels, filenames, distances, metric_label):
    k = len(indices)
    fig, axs = plt.subplots(1, k + 1, figsize=(15, 5))

    # Immagine query
    axs[0].imshow(cv2.cvtColor(cv2.imread(query_path), cv2.COLOR_BGR2RGB))
    axs[0].set_title("Query")
    axs[0].axis('off')

    # Immagini simili
    for i, idx in enumerate(indices):
        img_path = os.path.join("Part1", labels[idx], filenames[idx])
        match_img = cv2.imread(img_path)
        axs[i + 1].imshow(cv2.cvtColor(match_img, cv2.COLOR_BGR2RGB))
        axs[i + 1].set_title(f"Rank {i+1}\n{metric_label}={distances[idx]:.2f}")
        axs[i + 1].axis('off')

    plt.tight_layout()
    plt.show()

Ricerca Immagini Simili - Euclidea / Coseno  -- Task 3

In questa funzione chiamiamo una solo distanza 

In [13]:
def find_k_similar(img_path, k, extractor, features, filenames, labels):
    query_feature = extractor.extract_feature(img_path)
    if query_feature is None:
        return

    query_feature = np.array(query_feature).reshape(1, -1)
    dist_euc = euclidean_distances(features, query_feature).flatten()
    dist_cos = cosine_distances(features, query_feature).flatten()

    top_k_idx_euc = np.argsort(dist_euc)[:k]
    top_k_idx_cos = np.argsort(dist_cos)[:k]

    print(f"\nTop {k} simili a {img_path}:")
    print("=== Euclide ===")
    for rank, idx in enumerate(top_k_idx_euc):
        print(f"{rank+1}. {filenames[idx]} | Classe: {labels[idx]} | Euclid: {dist_euc[idx]:.4f}")

    print("=== Coseno ===")
    for rank, idx in enumerate(top_k_idx_cos):
        print(f"{rank+1}. {filenames[idx]} | Classe: {labels[idx]} | Cosine: {dist_cos[idx]:.4f}")

    # Visualizza entrambi
    plot_matches(img_path, top_k_idx_euc, labels, filenames, dist_euc, "Euclid")
    plot_matches(img_path, top_k_idx_cos, labels, filenames, dist_cos, "Cosine")

Ricerca Immagini Simili - Mahalanobis -- Task 3

In [14]:
def find_k_similar_mahalanobis(img_path, k, extractor, features, filenames, labels):
    query_feature = extractor.extract_feature(img_path)
    if query_feature is None:
        return

    query_feature = np.array(query_feature)
    cov = np.cov(features.T)
    try:
        cov_inv = np.linalg.inv(cov)
    except np.linalg.LinAlgError:
        print("[ERRORE] Covarianza non invertibile. Uso pseudoinversa.")
        cov_inv = np.linalg.pinv(cov)

    distances = np.array([mahalanobis(query_feature, f, cov_inv) for f in features])
    top_k_idx = np.argsort(distances)[:k]

    print(f"\nTop {k} simili (Mahalanobis) a {img_path}:")
    for rank, idx in enumerate(top_k_idx):
        print(f"{rank+1}. {filenames[idx]} | Classe: {labels[idx]} | Distanza: {distances[idx]:.2f}")

    plot_matches(img_path, top_k_idx, labels, filenames, distances, "M")

Esecuzione

In [ ]:
# === Parametri ===
base_folder = "Part1"
subfolders = ["brain_glioma", "brain_menin", "brain_tumor"]
output_file = "resnet1024.npz"

# === Estrai e salva features (solo una volta) ===
# process_and_save_features(base_folder, subfolders, output_file)

# === Carica features ===
features, filenames, labels = load_features(output_file)

# === Inizializza extractor ===
extractor = ResNetAvgPool1024Extractor()

# === Query ===
query_img = "Part1/brain_glioma/brain_glioma_0005.jpg"
find_k_similar(query_img, k=5, extractor=extractor, features=features, filenames=filenames, labels=labels)
find_k_similar_mahalanobis(query_img, k=5, extractor=extractor, features=features, filenames=filenames, labels=labels)